In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

In [3]:
raw_text = open('cien sonetos de amor.txt').readlines()

In [4]:
raw_text[:20]

['Soneto I \n',
 '\n',
 'Matilde, nombre de planta o piedra o vino, \n',
 'de lo que nace de la tierra y dura, \n',
 'palabra en cuyo crecimiento amanece, \n',
 'en cuyo estío estalla la luz de los limones. \n',
 'En ese nombre corren navíos de madera \n',
 'rodeados por enjambres de fuego azul marino, \n',
 'y esas letras son el agua de un río \n',
 'que desemboca en mi corazón calcinado. \n',
 'Oh nombre descubierto bajo una enredadera \n',
 'como la puerta de un túnel desconocido \n',
 'que comunica con la fragancia del mundo! \n',
 'Oh invádeme con tu boca abrasadora, \n',
 'indágame, si quieres, con tus ojos nocturnos, \n',
 'pero en tu nombre déjame navegar y dormir.\n',
 '\n',
 ' \n',
 '\n',
 'Soneto II \n']

In [5]:
raw_text_no_cr = []
for i, line in enumerate(raw_text):
    if line != '\n' and line!=' \n':
        if line[:7] == 'Soneto ':
            if i>0: raw_text_no_cr.append(' END_SONETO ')
            raw_text_no_cr.append(' START_SONETO ')
        else:
            raw_text_no_cr.append(line.replace(' \n', '\n')
                                  .replace('«', '"')
                                  .replace('»', '"')
                                  .replace('-', 'char_guion_medio')
                                  .replace('(', 'char_open_parent')
                                  .replace(')', 'char_close_parent')
                                  .replace('\n', ' char_new_line ')
                                  .replace('.', ' char_dot ')
                                  .replace(',', ' char_comma ')
                                  .replace('¡', ' char_exclamation_open ')
                                  .replace('!', ' char_exclamation_close ')
                                  .replace('¿', ' char_question_open ')
                                  .replace('?', ' char_question_close ')
                                  .replace(':', ' char_dos_puntos ')
                                  .replace('"', ' char_comillas ')
                                 )

In [6]:
raw_text_no_cr[:50]

[' START_SONETO ',
 'Matilde char_comma  nombre de planta o piedra o vino char_comma  char_new_line ',
 'de lo que nace de la tierra y dura char_comma  char_new_line ',
 'palabra en cuyo crecimiento amanece char_comma  char_new_line ',
 'en cuyo estío estalla la luz de los limones char_dot  char_new_line ',
 'En ese nombre corren navíos de madera char_new_line ',
 'rodeados por enjambres de fuego azul marino char_comma  char_new_line ',
 'y esas letras son el agua de un río char_new_line ',
 'que desemboca en mi corazón calcinado char_dot  char_new_line ',
 'Oh nombre descubierto bajo una enredadera char_new_line ',
 'como la puerta de un túnel desconocido char_new_line ',
 'que comunica con la fragancia del mundo char_exclamation_close  char_new_line ',
 'Oh invádeme con tu boca abrasadora char_comma  char_new_line ',
 'indágame char_comma  si quieres char_comma  con tus ojos nocturnos char_comma  char_new_line ',
 'pero en tu nombre déjame navegar y dormir char_dot  char_new_line ',


In [7]:
all_text = ''.join(raw_text_no_cr)
print(all_text[:200])

 START_SONETO Matilde char_comma  nombre de planta o piedra o vino char_comma  char_new_line de lo que nace de la tierra y dura char_comma  char_new_line palabra en cuyo crecimiento amanece char_comma


In [8]:
from sklearn.feature_extraction.text import CountVectorizer

In [9]:
vectorizer = CountVectorizer(stop_words=None, token_pattern='(?u)\\b\\w+\\b')

In [10]:
matrix = vectorizer.fit_transform([all_text])

In [11]:
len(vectorizer.vocabulary_)

2568

In [12]:
import re

In [13]:
separators = re.compile('(?u)\\b\\w+\\b').split(all_text)

In [14]:
for s in separators:
    if s!=' ' and s!='  ':
        print(s)

In [15]:
words = re.findall('(?u)\\b\\w+\\b', all_text)

In [16]:
words[:5]

['START_SONETO', 'Matilde', 'char_comma', 'nombre', 'de']

In [17]:
coded_words = [vectorizer.vocabulary_[word.lower()] for word in words]

In [18]:
coded_words[:10]

[2201, 1458, 425, 1608, 660, 1789, 1627, 1769, 1627, 2502]

In [19]:
len(set(coded_words))

2568

In [20]:
from keras.utils import to_categorical

Using TensorFlow backend.


In [21]:
to_categorical(coded_words).shape

(13990, 2568)

In [64]:
window = 10
step = 1
sentences = []
next_chars = []
for i in range(0, len(coded_words) - window, step):
    sentences.append(coded_words[i: i + window])
    next_chars.append(coded_words[i + window])

In [65]:
x_train = np.array(sentences)
y_train = to_categorical(np.array(next_chars))

In [66]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

In [67]:
x_train.shape

(13980, 10)

In [68]:
y_train.shape

(13980, 2568)

In [69]:
len(set(coded_words))

2568

In [131]:
vectorizer.get_feature_names()[max(coded_words)]

'única'

In [70]:
from models import get_model_1

In [71]:
model1 = get_model_1(len(set(coded_words)))
model1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 64)          164352    
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               66000     
_________________________________________________________________
dense_4 (Dense)              (None, 2568)              259368    
Total params: 489,720
Trainable params: 489,720
Non-trainable params: 0
_________________________________________________________________


In [72]:
model1.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [95]:
model1.fit(x_train, y_train, batch_size=256, epochs=400, verbose = 1)

In [74]:
initial_phrase = ['start_soneto', 'la', 'tarde', 'brilla' ,'en']
#initial_phrase = ['start_soneto', 'el', 'amor', 'es' ,'el']

In [75]:
predicted_words = [vectorizer.vocabulary_[word] for word in initial_phrase]
for i in range(200):
    pred_word = model1.predict_classes(np.array(predicted_words)[-5:].reshape(1,5))[0]
    if (vectorizer.vocabulary_['end_soneto'] == pred_word):
        break
    predicted_words.append(pred_word)

In [76]:
print(' ', end='')
for word_idx in predicted_words:
    decoded_word = vectorizer.get_feature_names()[word_idx]
    if decoded_word!='start_soneto':
        print(decoded_word.replace('char_comma', '\b,').
              replace('char_dot', '\b.').
              replace('char_dos_puntos', '\b:').
              replace('char_new_line', '\n')
              +
              ' ', end='')

 la tarde brilla en la noche en el agua de la piedra 
 nos plena vez con la sombra, 
 tu boca con la tierra, 
 sin vez con un cielo, 
 tu boca con la tierra, 
 sin vez con un cielo, 
 tu boca con la tierra, 
 sin vez con un cielo, 
 tu boca con la tierra, 
 sin vez con un cielo, 
 tu boca con la tierra, 
 sin vez con un cielo, 
 tu boca con la tierra, 
 sin vez con un cielo, 
 tu boca con la tierra, 
 sin vez con un cielo, 
 tu boca con la tierra, 
 sin vez con un cielo, 
 tu boca con la tierra, 
 sin vez con un cielo, 
 tu boca con la tierra, 
 sin vez con un cielo, 
 tu boca con la tierra, 
 sin vez con un cielo, 
 tu boca con la tierra, 
 sin vez con un cielo, 
 tu boca con la tierra, 
 sin vez con un cielo, 
 tu 

In [77]:
model1.save_weights('model_1-window-10.hdf5')

In [376]:
from models import get_model_1

In [378]:
model_1 = get_model_1(len(set(coded_words)))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, None, 64)          164352    
_________________________________________________________________
lstm_9 (LSTM)                (None, 100)               66000     
_________________________________________________________________
dense_8 (Dense)              (None, 2568)              259368    
Total params: 489,720
Trainable params: 489,720
Non-trainable params: 0
_________________________________________________________________


In [380]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (None, None, 10)          25680     
_________________________________________________________________
lstm_8 (LSTM)                (None, 100)               44400     
_________________________________________________________________
dense_7 (Dense)              (None, 2568)              259368    
Total params: 329,448
Trainable params: 329,448
Non-trainable params: 0
_________________________________________________________________


In [88]:
words_by_soneto = []
accum = []
for word in words:
    if word != 'END_SONETO':
        accum.append(word)
    else:
        words_by_soneto.append(accum)
        accum = []

In [90]:
sonetos_word_frec = []
for soneto in words_by_soneto:
    sonetos_word_frec.append(len(soneto))

In [94]:
max(sonetos_word_frec), min(sonetos_word_frec)

(170, 110)

In [101]:
coded_sonetos = []
for soneto in words_by_soneto:
    coded_sonetos.append([vectorizer.vocabulary_[word.lower()] for word in soneto])

In [119]:
def window_transform_codes_padding_mode(codes):
    len_out = len(codes)
    
    #output_start[-1] = text[:1]
    #output_start[len_out-1:len_out-2] = text[0:1]
    X = []
    y = []
    for i in range(len_out):
        #X.append()
        output_start = [-1]*(len_out-1)
        output_start[len_out-i-1:len_out] = codes[0:i]
        X.append(output_start)
        y.append(codes[i])
    return X, y

In [121]:
X_train, y_train = window_transform_codes_padding_mode(coded_sonetos[0])